In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [5]:
user = pd.read_csv('./data/raw/customers.csv')
item = pd.read_csv('./data/raw/articles.csv')
trans = pd.read_csv('./data/raw/transactions_train_all.csv')

In [6]:
trans.head(2)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2


In [7]:
trans['t_dat'] = pd.to_datetime(trans['t_dat'])
last_date = trans['t_dat'].max()
first_date = trans['t_dat'].min()

In [8]:
(first_date,last_date)

(Timestamp('2018-09-20 00:00:00'), Timestamp('2020-09-22 00:00:00'))

Use data from September 15, 2020 to September 22, for testing;
August 15, 2020 to September 15 for training; 
September 1,2019 to August 15,2020 for auxiliary statistical feature construction without participating in training.

In [9]:
trans2 = trans[(trans['t_dat'] >= '2019-09-01') & (trans['t_dat'] <= '2020-09-30')]

In [11]:
trans2['item_id'] = trans2['article_id']
trans2_grouped = trans2.groupby('customer_id')['article_id'].agg(list).reset_index()
trans2 = trans2.drop('article_id', axis=1).merge(trans2_grouped, on='customer_id')

/var/folders/t0/v5mt2fvx40j4y3x7fqgq6kg00000gn/T/ipykernel_87546/342743874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans2['item_id'] = trans2['article_id']


In [13]:
trans2['count'] = trans2['article_id'].apply(lambda x: len(x))

In [16]:
trans2['count'].describe()

count    1.567043e+07
mean     4.930410e+01
std      5.813617e+01
min      1.000000e+00
25%      1.500000e+01
50%      3.200000e+01
75%      6.200000e+01
max      1.067000e+03
Name: count, dtype: float64

In [18]:
trans3 = trans2[(trans2['count']>=6) & (trans2['count']<=100)]

In [20]:
trans3

,t_dat,customer_id,price,sales_channel_id,item_id,article_id,count
0,2019-09-01,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.016932,1,727880001,"[727880001, 767869001, 717490010, 547780022, 8...",24
1,2019-09-01,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1,767869001,"[727880001, 767869001, 717490010, 547780022, 8...",24
2,2019-09-01,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.008458,1,717490010,"[727880001, 767869001, 717490010, 547780022, 8...",24
3,2019-09-01,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.025407,1,547780022,"[727880001, 767869001, 717490010, 547780022, 8...",24
4,2019-11-16,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,0.022017,1,817953001,"[727880001, 767869001, 717490010, 547780022, 8...",24
...,...,...,...,...,...,...,...
15670405,2020-09-22,fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...,0.033881,2,867969003,"[867969008, 562245001, 867969003, 682550002, 5...",7
15670406,2020-09-22,fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...,0.033881,2,682550002,"[867969008, 562245001, 867969003, 682550002, 5...",7
15670407,2020-09-22,fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...,0.023712,2,562245018,"[867969008, 562245001, 867969003, 682550002, 5...",7
15670408,2020-09-22,fe99a0069d6b3c64c2707d0ce53b9311540917471d82df...,0.042356,2,816598018,"[867969008, 562245001, 867969003, 682550002, 5...",7


In [30]:
trans4 = trans3[(trans3['t_dat']>'2020-08-15') & (trans3['t_dat']<='2020-09-15')]
trans5 = trans3[(trans3['t_dat']>'2020-09-15')]


In [31]:
len(trans4)

920635

In [32]:
len(trans5)

190406

In [33]:
trans5.groupby('t_dat').count()

,customer_id,price,sales_channel_id,item_id,article_id,count
t_dat,,,,,,
2020-09-16,21489,21489,21489,21489,21489,21489
2020-09-17,31796,31796,31796,31796,31796,31796
2020-09-18,30919,30919,30919,30919,30919,30919
2020-09-19,29900,29900,29900,29900,29900,29900
2020-09-20,24935,24935,24935,24935,24935,24935
2020-09-21,25775,25775,25775,25775,25775,25775
2020-09-22,25592,25592,25592,25592,25592,25592


In [35]:
use_item = set(trans3.item_id.to_list())
use_user = set(trans3.customer_id.to_list())

In [37]:
len(use_user),len(use_item)

(594381, 70941)

In [43]:
item.head(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [41]:
user2 = user[user['customer_id'].apply(lambda x:x in use_user)]

In [44]:
item2 = item[item['article_id'].apply(lambda x:x in use_item)]

In [46]:
len(user2),len(item2)

(594381, 70941)

In [47]:
user2.to_csv('./dataset/user.csv', index=False)
item2.to_csv('./dataset/item.csv', index=False)
trans3.to_csv('./dataset/transaction.csv',index=False)